In [1]:
from simulation_numerical_vs_SW import *
import matplotlib.pyplot as plt
import numpy as np
import matplotlib

#plt.rc('text', usetex=True)

plt.rc('font', family='serif')
plt.rc('axes', titlesize=18)     # fontsize of the axes title
plt.rc('axes', labelsize=18)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=16)    # fontsize of the tick labels
plt.rc('ytick', labelsize=16)    # fontsize of the tick labels
plt.rc('legend', fontsize=16)    # legend fontsize#
# Transparent background for figures but keep the legend background white but its text color black
plt.rcParams['legend.facecolor'] = 'white'
plt.rcParams['legend.edgecolor'] = 'none'
plt.rcParams['legend.framealpha'] = 0.8
plt.rcParams['legend.shadow'] = False
plt.rcParams['figure.facecolor'] = 'none'
plt.rcParams['axes.facecolor'] = 'none'
# White spins, axis and ticks 
plt.rcParams['text.color'] = 'black'
plt.rcParams['axes.labelcolor'] = 'white'
plt.rcParams['xtick.color'] = 'white'
plt.rcParams['ytick.color'] = 'white'
plt.rcParams['axes.labelcolor'] = 'white'
plt.rcParams['axes.edgecolor'] = 'white'


In [2]:
simulation_params = {
    "Bz": 0.1,
    "M": 1000000.0,
    "L": 50,
    "Delta z": 150,
    "lx": 60,
    "ly": 60
}

In [ ]:
R0 = 50
skyrmion = CylindricalMagnet(M = simulation_params['M'], R = R0, L = simulation_params['L'], Delta_z = simulation_params['Delta z'])
H_system = QubitHamiltonianSkyrmion(lx=simulation_params['lx'], ly=simulation_params['ly'], CyMagnet=skyrmion, Bz=simulation_params['Bz'], N_boson_states=3)
H_system.prepare_symbolic_hamiltonian()
H_system.compute_numerical_simulation_values()

/home/leander/PhD/Python/Qubit_Skyrmions/simulation_numerical_vs_SW.py:103: RuntimeWarning: divide by zero encountered in scalar divide
  theta_val = 1/2 * np.atan(-(2 / self.kB * np.sqrt(1 / (Kp_val*Km_val*self.ky)))/((self.ky**2 - 1) / self.ky**2))


In [4]:
lx_lEs = np.linspace(0.5, 1.0, 5)[::-1]

similarities = []

for lx_lE in lx_lEs:
    print(f'lx/lE = {lx_lE}')
    H_system.add_drive(lx_lE)
    H_system.eff_lab_frame()
    H_system.eff_co_moving_frame()
    H_system.numerical_hamiltonian()
    H_system.drive_subs

    f = float((H_system.lab_frame_qubit['Rabi_freq']) / (2 * np.pi))
    T = 1 / f
    N_steps = 9000
    t_list = np.linspace(0, T, N_steps)

    numerical_results = mesolve([H_system.numerical_qubit['H0'], H_system.numerical_qubit['HDrive']], H_system.numerical_qubit['psi0'], e_ops=H_system.numerical_qubit['Projectors'], tlist=t_list, options={'progress_bar': 'enhanced', 'nsteps': N_steps})
    lab_frame_results = mesolve([H_system.lab_frame_qubit['H0'], H_system.lab_frame_qubit['HDrive']], H_system.lab_frame_qubit['psi0'], e_ops=H_system.lab_frame_qubit['Projectors'], tlist=t_list, options={'progress_bar': 'enhanced', 'nsteps': N_steps})
    co_frame_results  = mesolve([H_system.co_moving_frame_qubit['H0']] + H_system.co_moving_frame_qubit['HDrive'], H_system.co_moving_frame_qubit['psi0'], e_ops=H_system.co_moving_frame_qubit['Projectors'], tlist=t_list, options={'progress_bar': 'enhanced', 'nsteps': N_steps})

    spectrum_lab_frame = np.abs(lab_frame_results.expect[0])**2
    spectrum_numerical = np.abs(numerical_results.expect[0])**2

    norm_lab_frame = np.sqrt(np.sum(np.abs(spectrum_lab_frame)**2))
    norm_numerical = np.sqrt(np.sum(np.abs(spectrum_numerical)**2))
    similarity = np.dot(spectrum_lab_frame.conjugate(), spectrum_numerical) / (norm_lab_frame * norm_numerical)
    similarities.append(similarity)


    fig, ax = plt.subplots()

    ax.plot(t_list, 1-np.sin(t_list * float(H_system.lab_frame_qubit['Rabi_freq']) / 2)**2, label='Lab frame RWA')
    ax.plot(t_list, lab_frame_results.expect[0], label='Lab frame')
    ax.plot(t_list, numerical_results.expect[0], label='Numerical')

    ax.set_title(f'$l_x / l_E= {lx_lE}$, similarity = ${similarity:.5f}$', color='white')
    ax.set_xlabel('$t$')
    ax.set_ylabel(r'$P(0)$')
    ax.legend()
    plt.show()


lx/lE = 1.0


AttributeError: 'QubitHamiltonianSkyrmion' object has no attribute 'drive_symbols'

In [ ]:
fig, ax = plt.subplots()
ax.scatter(lx_lEs, similarities)
ax.set_title('Similarity between numerical and lab frame results', color='white')
ax.set_xlabel('$l_x / l_E$')
